In [12]:
import PyPDF2
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

SyntaxError: invalid syntax (4103738896.py, line 4)

In [6]:
#Converting PDF to TXT
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    with open('pdf2text.txt', 'w', encoding='utf-8') as file:
        file.write(text)
    return 'pdf2text.txt'

In [9]:
# 1. Load and Embed Documents
pdf2text = extract_text_from_pdf('./loi-n-01-00-portant-organisation-de-lenseignement-supérieur.pdf')
loader = TextLoader('./'+pdf2text) # Replace with your document source
documents = ['pdf2text.txt']

In [11]:
print(torch.__version__)

2.5.0+cpu


loading the tokenizer and the embedding using hugginface models

In [4]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
embeddings = []
for doc in documents:
    inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Get the mean pooling of the embeddings (average over tokens)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings.append(embedding)

embeddings = np.vstack(embeddings)

NameError: name 'np' is not defined